# Description

Notebook for running Herlands et. al's algorithm against our simulation scenarios. Forked from their `real_data_general.ipynb` notebook.

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import os
import math
import time
import statsmodels.formula.api as smf  # for doing statistical regression
import statsmodels.api as sm
import statsmodels.discrete.discrete_model as smd

from data_functions import *
from subset_functions import *
from model_functions import *
from search_functions import *
from analysis_functions import *

import matplotlib.pyplot as plt
#% matplotlib inline

dir_results = '../results/'

%load_ext autoreload
%autoreload 2

In [2]:
def real_data_RDSS(dir_data, file_json, subsample, plotting_data, verbose_data, obs_model, data_type, f_base, k, verbose_search, plotting_search):

    seed = 43
    np.random.seed(seed)

    # Get data
    x, y, z, T, x_cols, inst, discont = data_real(dir_data, file_json, subsample=subsample, plotting=plotting_data, verbose=verbose_data)

    # Search for discontinuity
    x, x_means, x_stds = normalize_xz(x, z)
    llrs, neighs, subsets_best, beta_0_n, beta_1_n, T_fx, llrs_n, llrs_a, centers_n, pivots_best, subset_imax = \
    RDSS_residual(obs_model, T, x, z, f_base=f_base, all_points=False, k=k, verbose=verbose_search, plotting=plotting_search)
    x = unnormalize_xz(x, z, x_means, x_stds)

    # Plot the best
    max_arg = np.argmax(llrs)
    plot_neigh(x, z, out=T, out_name='Top subset', neigh=[subset_neigh(neighs[:,max_arg], subsets_best[max_arg]),subset_neigh(neighs[:,max_arg], ~subsets_best[max_arg])])
    plt.hist(llrs, bins=100); plt.show()
    
    # randomization testing
    iters_rand = 20
    alpha = 0.05
    k_samples = k

    T_hat_master = get_pred_mean(T_fx, f_base, x)

    llr_sig, llr_max_samples, llr_all_samples = rand_testing(obs_model, T_fx, T_hat_master,
                                            k_samples, iters_rand, 
                                            alpha, T, x, z, f_base,
                                            all_points=False, k=[k])

    plot_neigh(x, z, llrs>llr_sig, 'LLR -- significant at alpha '+str(alpha))
    
    return llrs, neighs, subsets_best, beta_0_n, beta_1_n, T_fx, llrs_n, llrs_a, centers_n, pivots_best, subset_imax, llr_sig, llr_max_samples, llr_all_samples

In [3]:
def get_sig_points(dir_data, file_json, subsample, plotting_data, verbose_data, obs_model, data_type, f_base, k, verbose_search, plotting_search, seed=42):
    np.random.seed(seed)

    # Get data
    x, y, z, T, x_cols, inst, discont = data_real(dir_data, file_json, subsample=subsample, plotting=plotting_data, verbose=verbose_data)

    # Search for discontinuity
    x, x_means, x_stds = normalize_xz(x, z)
    llrs, neighs, subsets_best, beta_0_n, beta_1_n, T_fx, llrs_n, llrs_a, centers_n, pivots_best, subset_imax = \
    RDSS_residual(obs_model, T, x, z, f_base=f_base, all_points=False, k=k, verbose=verbose_search, plotting=plotting_search)
    x = unnormalize_xz(x, z, x_means, x_stds)

    # Plot the best
    #max_arg = np.argmax(llrs)
    #plot_neigh(x, z, out=T, out_name='Top subset', neigh=[subset_neigh(neighs[:,max_arg], subsets_best[max_arg]),subset_neigh(neighs[:,max_arg], ~subsets_best[max_arg])])
    #plt.hist(llrs, bins=100); plt.show()
    
    # randomization testing, multiple test correction
    iters_rand = 20
    alpha = 0.05 / k 
    k_samples = k
    print(alpha)
    
    T_hat_master = get_pred_mean(T_fx, f_base, x)

    llr_sig, llr_max_samples, llr_all_samples = rand_testing(obs_model, T_fx, T_hat_master,
                                            k_samples, iters_rand, 
                                            alpha, T, x, z, f_base,
                                            all_points=False, k=[k])

    #plot_neigh(x, z, llrs>llr_sig, 'LLR -- significant at alpha '+str(alpha))
    x, y, z, T, x_cols, inst, discont = data_real(dir_data, file_json, subsample=subsample, plotting=plotting_data, verbose=verbose_data)
    
    return x[llrs > llr_sig]

In [4]:
c1 = 0.25
c2 = 0.75

In [5]:
def run_sim(gap, seed):
    """
    Runs simulation of Herlands for a given gap and seed.
    
    Returns:
        (bool, bool, int): c1 in cutpoints, c2 in cutpoints, number of significant points
    """
    dir_data = 'blend_sim'
    file_json = f"blend_sim_gap{gap}_seed{seed}.json"
    
    meta_dict = {"file": [f"gap{gap}_seed{seed}.csv"], 
                 "x": ["x", "covar"], 
                 "z": ["x", "covar"], 
                 "T": ["t"], 
                 "y": ["p"]}
    
    # migrate prep_real_data.py functionality to here
    with open(f"../data/{dir_data}/{file_json}", "w") as f:
        json.dump(meta_dict, f)
    
    #subsample = 100 # for testing
    subsample = False

    plotting_data = False
    verbose_data = False

    # Search
    obs_model = 'normal'    # {'normal', 'bernoulli'}
    data_type ='binary'        # {'cont', 'binary'}
    f_base    = 'Logit_poly1'  # {'OLS_poly1', 'Logit_poly1',
    k = 100
    verbose_search = False
    plotting_search = False

    sig_x = get_sig_points(dir_data, file_json, subsample, plotting_data, verbose_data, obs_model, data_type, f_base, k, verbose_search, plotting_search, seed=seed)
    
    sig_x = np.round(sig_x, 2)
    #print(sig_x)
    return c1 in list(sig_x[0]), c2 in list(sig_x[0]), sig_x.shape[0], sig_x[0]
    

In [ ]:
%%time
%%capture
import multiprocessing
import contextlib

gaps = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
starts = [0, 100, 200, 300, 400]

out_path = "{0}seed{1}/herlands_results_gap{2}.pkl"

for start in starts:
    seeds = range(start, start+100)
    for gap in gaps:
        args = [(gap, seed) for seed in seeds]
        with multiprocessing.Pool(20) as pool:
            #with open(os.devnull, "w") as f, contextlib.redirect_stderr(f):
            results = pool.starmap(run_sim, args)
            pickle.dump(results, open(out_path.format(dir_results, start, gap), "wb"), -1)